# Prerequisite

When u are not familiar with -> `pip install -r requirements.txt`, use this codeblock for installing the required packages

In [1]:
# !pip install jupyter
# !pip install scikit-learn
# !pip install pandas
# !pip install numpy

In [2]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Functions for evaluation purposes

In [3]:
def resultClassifierfloat(row):
    threshold = 0.5
    if (row['prediction'] > threshold and row['label'] == True):
        return 'TP'
    if (row['prediction'] < threshold and row['label'] == False):
        return 'TN'
    if (row['prediction'] < threshold and row['label'] == True):
        return 'FN'
    if (row['prediction'] > threshold and row['label'] == False):
        return 'FP'


def resultClassifierint(row):
    if (row['label'] == row['prediction'] and row['label'] == True):
        return 'TP'
    if (row['label'] == row['prediction'] and row['label'] == False):
        return 'TN'
    if (row['label'] != row['prediction'] and row['label'] == True):
        return 'FN'
    if (row['label'] != row['prediction'] and row['label'] == False):
        return 'FP'


def evaluation(classifier, name, X_test, y_test):
    # Predict the response for test dataset
    y_pred = classifier.predict(X_test)
    results = pd.DataFrame({'label': y_test.values, 'prediction': y_pred}, index=y_test.index)
    results['confusion_matrix'] = results.apply(resultClassifierint, axis=1)
    results_counts = results['confusion_matrix'].value_counts()

    print(name)
    print(results_counts)
    print('Precision: ', (results_counts.loc['TP']) / (results_counts.loc['TP'] + results_counts.loc['FP']))
    print('Recall: ', (results_counts.loc['TP']) / (results_counts.loc['TP'] + results_counts.loc['FN']))
    print("Accuracy: ", metrics.accuracy_score(y_test, y_pred))
    print(metrics.classification_report(y_test, y_pred))

# Load preprocessed data set

Split into train, test etc

In [4]:
trainset = pd.read_csv('./final_trainset.csv')
trainset.iloc[52283:, trainset.columns.get_loc('match')] = 0
trainset1 = trainset[:52283]
trainset2 = trainset[52283:]

train_test_set1 = trainset1[:41581]
train_test_set2 = trainset2[:41581]
train_test_set = pd.concat([train_test_set1, train_test_set2], axis=0, join='inner')

validation_set1 = trainset1[41581:]
validation_set2 = trainset2[41581:]
validation_set = pd.concat([validation_set1, validation_set2], axis=0, join='inner')

In [5]:
feature_cols = ['date_binary',
                'jac_total',
                'title_similarity',
                'content_similarity',
                'sleutelwoorden_lenmatches',
                'BT_TT_lenmatches',
                'title_no_stop_lenmatches',
                '1st_paragraph_no_stop_lenmatches',
                'numbers_lenmatches']
X = train_test_set[feature_cols]
X = X.fillna(0)
y = train_test_set['match']

In [6]:
X[X.isna()].sum().sum()

np.float64(0.0)

In [7]:
len(X[~X.isna()]) - len(X[X.isna()])

0

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Traininig Phase
```
# Note ->  Changed in version 1.1: The default of max_features changed from "auto" to "sqrt".
```

In [9]:
rf = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                            max_depth=40, max_features='sqrt', max_leaf_nodes=None,
                            min_impurity_decrease=0.0,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=150,
                            n_jobs=-1, oob_score=False, random_state=None,
                            verbose=0, warm_start=False)

In [10]:
rf

RandomForestClassifier(bootstrap=False, max_depth=40, n_estimators=150,
                       n_jobs=-1)

In [11]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=False, max_depth=40, n_estimators=150,
                       n_jobs=-1)

# Evaluation Phase
### -> given test data

In [12]:
evaluation(rf, 'default_grid_forest', X_test, y_test)

default_grid_forest
confusion_matrix
TP    11891
TN    11690
FP      708
FN      660
Name: count, dtype: int64
Precision:  0.9438050638939598
Recall:  0.9474145486415425
Accuracy:  0.945168143011744
              precision    recall  f1-score   support

           0       0.95      0.94      0.94     12398
           1       0.94      0.95      0.95     12551

    accuracy                           0.95     24949
   macro avg       0.95      0.95      0.95     24949
weighted avg       0.95      0.95      0.95     24949



### -> given validation data

In [13]:
feature_cols = ['date_binary',
                'jac_total',
                'title_similarity',
                'content_similarity',
                'sleutelwoorden_lenmatches',
                'BT_TT_lenmatches',
                'title_no_stop_lenmatches',
                '1st_paragraph_no_stop_lenmatches',
                'numbers_lenmatches']

X = validation_set[feature_cols]
X = X.fillna(0)
y = validation_set['match']
x_validation = X
y_validation = y

evaluation(rf, 'default_grid_forest', x_validation, y_validation)

default_grid_forest
confusion_matrix
TP    10140
TN     9207
FP      881
FN      562
Name: count, dtype: int64
Precision:  0.9200617003901642
Recall:  0.9474864511306298
Accuracy:  0.9305916305916306
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     10088
           1       0.92      0.95      0.93     10702

    accuracy                           0.93     20790
   macro avg       0.93      0.93      0.93     20790
weighted avg       0.93      0.93      0.93     20790



# Save model weights

In [14]:
import pickle

filename = 'rf_model_pretrained_11_11_24.pkl'
pickle.dump(rf, open(filename, 'wb'))

# Load pretrained model weights

In [15]:
loaded_model = pickle.load(open(filename, 'rb'))

# Redo evaluation

### -> test data

In [16]:
evaluation(loaded_model, 'default_grid_forest', X_test, y_test)

default_grid_forest
confusion_matrix
TP    11891
TN    11690
FP      708
FN      660
Name: count, dtype: int64
Precision:  0.9438050638939598
Recall:  0.9474145486415425
Accuracy:  0.945168143011744
              precision    recall  f1-score   support

           0       0.95      0.94      0.94     12398
           1       0.94      0.95      0.95     12551

    accuracy                           0.95     24949
   macro avg       0.95      0.95      0.95     24949
weighted avg       0.95      0.95      0.95     24949



### -> validation data

In [17]:
evaluation(loaded_model, 'default_grid_forest', x_validation, y_validation)

default_grid_forest
confusion_matrix
TP    10140
TN     9207
FP      881
FN      562
Name: count, dtype: int64
Precision:  0.9200617003901642
Recall:  0.9474864511306298
Accuracy:  0.9305916305916306
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     10088
           1       0.92      0.95      0.93     10702

    accuracy                           0.93     20790
   macro avg       0.93      0.93      0.93     20790
weighted avg       0.93      0.93      0.93     20790

